# Test LLM2Vec
* Simple Prototype for LLM endcoding

In [1]:
import torch
from llm2vec import LLM2Vec

print("Loading model...")
model = LLM2Vec.from_pretrained(
    "McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp",
    peft_model_name_or_path="McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp-supervised",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)


Loading model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
from typing import Callable, Deque, Dict, Iterable, Iterator, List, Optional, Set, Tuple
import numpy as np

serial1: dict[int, str] = {}
serial1[123] = "hello world"
serial1[456] = "goodbye world"

serial2: dict[int, str] = {}
serial2[223] = "hello world"
serial2[789] = "goodbye world"

serializations = [serial1, serial2]

merged_serializations: dict[int, str] = {}
for s in serializations:
    merged_serializations.update(s)

def encode_texts(texts: List[str]) -> List[np.ndarray]:
    return np.array(model.encode(texts, batch_size=32))
merged_encodings = encode_texts(list(merged_serializations.values())) 


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


AttributeError: 'LlamaBiModel' object has no attribute 'rotary_emb'

In [7]:
import torch
from llm2vec import LLM2Vec
import numpy as np

patients = [
    'The patient is a 23 year-old male of caucasian race.',
    'The patient is a 32 year-old male of caucasian race.',
    'The patient is a 83 year-old female of black race.',
]   


model = LLM2Vec.from_pretrained(
    "McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp",
    peft_model_name_or_path="McGill-NLP/LLM2Vec-Meta-Llama-3-8B-Instruct-mntp-supervised",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

patients_encoded = np.array(model.encode(patients, batch_size=1))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/sc-projects/sc-proj-dh-ag-eils-ml/shared_envs/EHRSHOT_ENV/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Batches: 100%|██████████| 3/3 [00:00<00:00,  3.45it/s]


RuntimeError: Could not infer dtype of numpy.int64

In [5]:
instruction = "Classify the following description of a patient as being at risk for death or not: "

def append_instruction(instruction, sentences):
    new_sentences = []
    for s in sentences:
        new_sentences.append([instruction, s, 0])
    return new_sentences

patients = [
    'The patient is a 23 year-old male of caucasian race.',
    'The patient is a 32 year-old male of caucasian race.',
    'The patient is a 83 year-old female of black race.',
]
patients = 1300 * patients

patients_instructions = append_instruction(instruction, patients)


In [6]:
import numpy as np
batch_size = 32

# patients_encoded = np.asarray(model.encode(patients, batch_size=batch_size))
patients_encoded = np.array(model.encode(patients, batch_size=batch_size))

# Compute cosine similarity
p_encoded_norm = torch.nn.functional.normalize(patients_encoded, p=2, dim=1)
cos_sim = torch.mm(p_encoded_norm, p_encoded_norm.transpose(0, 1))
print(cos_sim)

Batches: 100%|██████████| 122/122 [00:14<00:00,  8.26it/s]


tensor([[1.0000, 0.8653, 0.4906,  ..., 1.0000, 0.8653, 0.4906],
        [0.8653, 1.0000, 0.4857,  ..., 0.8653, 1.0000, 0.4857],
        [0.4906, 0.4857, 1.0000,  ..., 0.4906, 0.4857, 1.0000],
        ...,
        [1.0000, 0.8653, 0.4906,  ..., 1.0000, 0.8653, 0.4906],
        [0.8653, 1.0000, 0.4857,  ..., 0.8653, 1.0000, 0.4857],
        [0.4906, 0.4857, 1.0000,  ..., 0.4906, 0.4857, 1.0000]])


In [2]:
# Encoding queries using instructions
instruction = (
    "Given a web search query, retrieve relevant passages that answer the query:"
)
queries = [
    [instruction, "how much protein should a female eat"],
    [instruction, "summit define"],
]
q_reps = l2v.encode(queries)

# Encoding documents. Instruction are not required for documents
documents = [
    "As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
    "Definition of summit for English Language Learners. : 1  the highest point of a mountain : the top of a mountain. : 2  the highest level. : 3  a meeting or series of meetings between the leaders of two or more governments.",
]
d_reps = l2v.encode(documents)

# Compute cosine similarity
q_reps_norm = torch.nn.functional.normalize(q_reps, p=2, dim=1)
d_reps_norm = torch.nn.functional.normalize(d_reps, p=2, dim=1)
cos_sim = torch.mm(q_reps_norm, d_reps_norm.transpose(0, 1))

print(cos_sim)
"""
tensor([[0.6470, 0.1619],
        [0.0786, 0.5844]])
"""

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.58it/s]

tensor([[0.6469, 0.1619],
        [0.0789, 0.5837]])


'\ntensor([[0.6470, 0.1619],\n        [0.0786, 0.5844]])\n'